## 事前準備

1. Google Cloud の新しいプロジェクトを作成します。
2. Cloud Shell を開いて、次のコマンドで API を有効化します。
```
gcloud services enable \
  aiplatform.googleapis.com \
  notebooks.googleapis.com \
  cloudresourcemanager.googleapis.com

```
3. 次のコマンドで Vertex AI Workbench のインスタンスを作成します。
```
PROJECT_ID=$(gcloud config list --format 'value(core.project)')
gcloud workbench instances create agent-development \
  --project=$PROJECT_ID \
  --location=us-central1-a \
  --machine-type=e2-standard-2
```

4. クラウドコンソールのナビゲーションメニューから「Vertex AI」→「Workbench」を選択すると、作成したインスタンス agent-develpment があります。インスタンスの起動が完了するのを待って、「JUPYTERLAB を開く」をクリックしたら、「Python 3(ipykernel)」の新規ノートブックを作成します。

5. この後は、ノートブック上で以下のコマンドを実行していきます。

## ADK パッケージのインストール

In [ ]:
%pip install --upgrade --user google-adk

インストールしたパッケージを利用可能にするために、次のコマンドでカーネルを再起動します。

再起動を確認するポップアップが表示されるので [Ok] をクリックします。

In [2]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## Serach Agent App の作成

In [1]:
import copy, datetime, json, os, pprint, time, uuid
import vertexai
from google.genai.types import Part, UserContent, ModelContent
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import google_search

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [2]:
instruction = '''
You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.
'''

search_agent = LlmAgent(
    name='search_agent',
    model='gemini-2.0-flash-001',
    description='Agent to answer questions using Google Search.',
    instruction=instruction,
    tools=[google_search]
)

In [3]:
class LocalApp:
    def __init__(self, agent, app_name, user_id):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._app_name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## 実行例

In [23]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

 高田馬場でおすすめのカレー屋はたくさんありますが、いくつかご紹介します。


 高田馬場にはたくさんのカレー屋さんがあり、それぞれに個性があっておすすめです。主なカレー店としては、以下のようなお店があります。

*   **カレーライス専門店ブラザー**： 鯖キーマカレーが人気のお店です。
*   **横浜ボンベイ 高田馬場店**： ボンベイカレーやカシミールカレーなど、独特なカレーが楽しめます。
*   **プネウマカレー**： コスパの良いカレーとして知られています。
*   **エチオピア 高田馬場店**： 野菜カレーが人気で、辛さが選べる本格的なカレー屋さんです。
*   **白カレーの店 1/f ゆらぎ**： その名の通り、白いカレーが特徴のお店です。

他にも、アプサラ、まっちゃんカレー、マイカリー食堂など、様々なカレー店が高田馬場にはあります。ぜひ色々試して、お好みのカレーを見つけてみてください。


SessionService が管理するセッション情報を確認します。

セッションを特定するのに必要な、`app_name`、`user_id`、`session_id` を確認します。

In [24]:
app_name = client._session.app_name
user_id = client._session.user_id
session_id = client._session.id

app_name, user_id, session_id

('Search Agent App', 'user00', '9888fd325f424ca7972e04235037b4a7')

確認した情報を使って、セッションに保存された情報を確認します。

2 つのイベント（ユーザーの質問とエージェントの回答）が記録されています。

エージェントの回答を生成する際に用いた Google Serch の情報なども確認できます。

In [25]:
session = client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

session

Session(id='9888fd325f424ca7972e04235037b4a7', app_name='Search Agent App', user_id='user00', state={}, events=[Event(content=UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-5f48e9d1-08a8-47b2-8830-2e277a22a7d5', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='05DJgw0i', timestamp=1747344623.65059), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text=

In [26]:
def format_timestamp(ts):
    return datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S.%f')

for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-15 21:30:23.650590 user
2025-05-15 21:30:23.651104 search_agent


続けて質問すると、セッション情報を踏まえた回答が得られます。

In [27]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

 家族連れにおすすめのカレー店としては、以下の点が考慮されるかと思います。

*   **プネウマカレー**：子供向けのサイズ展開があり、小盛もあるため、子供とシェアしやすいです。離乳食の持ち込みも可能で、土曜日には子供連れも来店するそうです。ただし、カウンター席のみなので、ある程度1人で座れる子供に向いています。テイクアウトも可能なので、家でゆっくり食べることもできます。
*   **アプサラ**：子供連れも歓迎しており、バナナリーフ包みカレーは混ぜて食べるのが子供の頃のカレーを思い出すという口コミもあります。
*   **フードウェイ 高田馬場店**：家族連れの利用が多く、子供も楽しめる雰囲気です。
*   **大戸屋ごはん処 高田馬場駅前店**：お子様野菜カレーがあり、小学生以下のお子様のみ注文できます。

食べログなどのグルメサイトでは、子連れで利用しやすいカレー店が多数紹介されています。これらのサイトを参考に、席の種類（テーブル席、ソファ席など）、子供向けメニューの有無、アレルギー対応などを考慮して選ぶと良いでしょう。


先ほどと同様にセッションに記録されたイベントを確認すると、新しい質問と回答のイベントが追加されています。

In [28]:
session = client._runner.session_service.get_session(
    app_name = app_name,
    user_id = user_id,
    session_id = session_id,
)

session

Session(id='9888fd325f424ca7972e04235037b4a7', app_name='Search Agent App', user_id='user00', state={}, events=[Event(content=UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-5f48e9d1-08a8-47b2-8830-2e277a22a7d5', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='05DJgw0i', timestamp=1747344623.65059), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text=

In [29]:
for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-15 21:30:23.650590 user
2025-05-15 21:30:23.651104 search_agent
2025-05-15 21:30:49.771242 user
2025-05-15 21:30:49.771849 search_agent


## システムインストラクションの確認

ちょっとした裏技を使って、LlmAgent オブジェクトが Gemini API に送っている生の情報を覗いてみます。

In [30]:
import pprint
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest

async def _run_one_step_async(
      self,
      invocation_context: InvocationContext,
  ) -> AsyncGenerator[Event, None]:
    """One step means one LLM call."""
    llm_request = LlmRequest()

    # Preprocess before calling the LLM.
    async for event in self._preprocess_async(invocation_context, llm_request):
        yield event
    if invocation_context.end_invocation:
        return

    # Calls the LLM.
    model_response_event = Event(
        id=Event.new_id(),
        invocation_id=invocation_context.invocation_id,
        author=invocation_context.agent.name,
        branch=invocation_context.branch,
    )

    ## DEBUG output
    if DEBUG:
        print('## Prompt contents ##')
        pprint.pp(llm_request.contents)
        print('----')
        print('## System instruction ##')
        print(llm_request.config.system_instruction)
        print('----')
        print('## Tools config ##')
        pprint.pp(llm_request.config.tools)
        print('----')
    ####

    async for llm_response in self._call_llm_async(
        invocation_context, llm_request, model_response_event
    ):
        # Postprocess after calling the LLM.
        async for event in self._postprocess_async(
            invocation_context, llm_request, llm_response, model_response_event
        ):
            yield event

from google.adk.flows.llm_flows.base_llm_flow import BaseLlmFlow
_run_one_step_async_original = copy.deepcopy(BaseLlmFlow._run_one_step_async)
BaseLlmFlow._run_one_step_async = _run_one_step_async

In [32]:
client = LocalApp(search_agent, 'Search Agent App', 'user00')

DEBUG = True
query = '''
高田馬場のおすすめのカレー屋は？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user')]
----
## System instruction ##

You are a friendly AI assistant that answers user's queries.
Use google_search to give answers based on the latest and objective information.


You are an agent. Your internal name is "search_agent".

 The description about you is "Agent to answer questions using Google Search."
----
## Tools config ##
[Tool(retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, enterprise_web_search=None, google_maps=None, code_execution=None, function_declarations=None)]
----
高田馬場でおすすめのカレー屋について調べてみました。


 高田馬場にはたくさんのカレー屋さんがありますね！おすすめのカレー屋さんをいくつかご紹介します。

*   **カレーライス専門店ブラザー:** 鯖キーマカレーが人気のお店です。
*   **エチオピアカリーキッチン 高田馬場店:** 長時間煮込んだ野菜が美味しいトロトロカレーが味わえます。
*   **横浜ボンベイ 高田馬場店:** サラッとしたルーが特徴で、カシミールカレーが人気です。
*

In [33]:
query = '''
特に家族連れにおすすめなのは？
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n高田馬場のおすすめのカレー屋は？\n')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='高田馬場でおすすめのカレー屋について調べてみました。\n\n'), Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text=' 高田馬場にはたくさんのカレー屋さんがありますね！おすすめのカレー屋さんをいくつかご紹介します。\n\n*   **カレーライス専門店ブラザー:** 鯖キーマカレーが人気のお店です。\n*   **エチオピアカリーキッチン 高田馬場店:** 長時間煮込んだ野菜が美味しいトロトロカレーが味わえます。\n*   **横浜ボンベイ 高田馬場店:** サラッとしたルーが特徴で、カシミールカレーが人気です。\n*   **プネウマカレー:** コスパ最強の人気店で、チキンカレーがおすすめです。\n*   **白カレーの店 1/f ゆらぎ:** その名の通り、白いカレーが楽しめるお店です。\n*   **マイカリー食堂 高田馬場店:** 様々な種類のカレーが楽しめるお店です。\n*   **ゴーゴ

`## Prompt contents ##` の部分を見ると、セッションに記録されたイベントをすべてプロンプトに含めるのではなく、コンテキスト（これまでの会話の流れ）を把握するのに必要な情報が選択されていることがわかります。

裏技で変更した内部関数を元に戻しておきます。

In [35]:
DEBUG = False
BaseLlmFlow._run_one_step_async = _run_one_step_async_original

## AgentEngine へのデプロイ

In [15]:
from vertexai import agent_engines

remote_agent = agent_engines.create(
    agent_engine=search_agent,
    requirements=[
        'google-cloud-aiplatform[adk,agent_engines]',
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.88.0'}
The following requirements are missing: {'cloudpickle'}
The following requirements are appended: {'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'cloudpickle==3.1.1']
Using bucket etsuji-15pro-poc
Wrote to gs://etsuji-15pro-poc/agent_engine/agent_engine.pkl
Writing to gs://etsuji-15pro-poc/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://etsuji-15pro-poc/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/879055303739/locations/us-central1/reasoningEngines/3271240606680088576/operations/3308747525283905536
View progress and logs at https://console.cloud.google.com/logs/query?project=etsuji-15pro-poc
AgentEngine created. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/327

In [16]:
class RemoteApp:
    def __init__(self, remote_agent, user_id):
        self._remote_agent = remote_agent
        self._user_id = user_id
        self._session = remote_agent.create_session(user_id=self._user_id)
    
    def _stream(self, query):
        events = self._remote_agent.stream_query(
            user_id=self._user_id,
            session_id=self._session['id'],
            message=query,
        )
        result = []
        for event in events:
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

    def stream(self, query):
        # Retry 4 times in case of resource exhaustion 
        for c in range(4):
            if c > 0:
                time.sleep(2**(c-1))
            result = self._stream(query)
            if result:
                return result
            if DEBUG:
                print('----\nRetrying...\n----')
        return None # Permanent error

In [17]:
remote_client = RemoteApp(remote_agent, 'user00')

query = '''
今年のゴールデンウィークは、カレンダー通りで何連休でしょうか？
'''
_ = remote_client.stream(query)

 今年のゴールデンウィークがカレンダー通りだとすると何連休になるか調べてみます。


 2025年のゴールデンウィークは、カレンダー通りの場合、5月3日（土）から5月6日（火・振替休日）までの4連休となります。ただし、4月29日（昭和の日）と5月3日～6日の間に平日が3日あります。これらの平日に休みを取ると、最大で11連休にすることも可能です.



In [18]:
query = '''
来年はどうなりますか？
'''
_ = remote_client.stream(query)

 来年のゴールデンウィークがカレンダー通りだとすると何連休になるか調べてみます。

 2026年のゴールデンウィークについてですが、カレンダー通りだと5連休になる見込みです. 具体的には、5月2日（土）から5月6日（水・振替休日）までが5連休となります.

ただし、4月29日（昭和の日）が水曜日で、その前後の平日を休むかどうかで、連休の日数は変わってきます. 例えば、4月30日（木）と5月1日（金）を休むと8連休、さらに4月27日と28日も休むと12連休も可能です.



セッション情報を参照する際は、`VertexAiSessionService` を使用します。

In [19]:
app_name = remote_client._session['app_name']
user_id = remote_client._session['user_id']
session_id = remote_client._session['id']

app_name, user_id, session_id

('3271240606680088576', 'user00', '1446126621118431232')

In [20]:
from google.adk.sessions import VertexAiSessionService

session_service = VertexAiSessionService(
    project = PROJECT_ID,
    location = LOCATION,
)

session = session_service.get_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id,
)

session

Session(id='1446126621118431232', app_name='3271240606680088576', user_id='user00', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='\n今年のゴールデンウィークは、カレンダー通りで何連休でしょうか？\n')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-d8189206-a65e-4cb5-9463-32145651420c', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='6146459665023631360', timestamp=1747344505.152209), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_respons

In [21]:
for event in session.events:
    print(format_timestamp(event.timestamp), event.author)

2025-05-15 21:28:25.152209 user
2025-05-15 21:28:25.215643 search_agent
2025-05-15 21:28:27.724802 user
2025-05-15 21:28:27.788258 search_agent


## 後片付け

デプロイ済みのエージェントをまとめて削除します。

In [37]:
for agent in agent_engines.list():
    print(agent.gca_resource.name)
    agent.delete(force=True)

projects/879055303739/locations/us-central1/reasoningEngines/3271240606680088576
Delete Agent Engine backing LRO: projects/879055303739/locations/us-central1/operations/4073233562030047232
Agent Engine deleted. Resource name: projects/879055303739/locations/us-central1/reasoningEngines/3271240606680088576
